In [1]:
import pandas as pd
import numpy
import pickle
import time
import joblib
import gensim
import matplotlib.pyplot as plt

from itertools import islice
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.corpora import Dictionary, bleicorpus
from gensim.matutils import hellinger
from gensim.models.coherencemodel import CoherenceModel

# Remove warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the dataset.
df = pd.read_pickle("/project/biocomplexity/sdad/projects_data/ncses/prd/Paper/FR_meta_and_final_tokens_23DEC21.pkl")
df.head()

# Compute the time variable
year = df['FY'].unique()
del df

In [11]:
# Find topics that maximise the coherence for each windows
path = '/project/biocomplexity/sdad/projects_data/ncses/prd/Dynamic_Topics_Modelling/nmf_fullabstract/'
n_topics = list(range(20,61,5))
model = []
coherence = []
max_coherence = []
    
for fy in year[0]:
    # upload the result that are necessary for the coherence
    (nmf_time,topics_list,W_list,H_list) = joblib.load( path+'nmf_out/windows_nmf2008.pkl' )
    del nmf_time
    del W_list
    del H_list
    
    (dictionary,corpus) = joblib.load( '/project/biocomplexity/sdad/projects_data/ncses/prd/Dynamic_Topics_Modelling/nmf_fullabstract/dico_corpus.pkl' )
    del corpus
    
    (tf_idf,tfidf_vectorizer,docs) = joblib.load( path+'Term_docs_2008.pkl' )
    del tf_idf
    del tfidf_vectorizer
    
    for t in range(0,len(n_topics)):
        term_rankings = topics_list[t]
        cm = CoherenceModel(topics=term_rankings, dictionary=dictionary, texts=docs, coherence='c_v', topn=10, processes=30)
        coherence.append(cm.get_coherence())
    
    # find the topics that maximize the coherence
    max_value = numpy.nanmax(coherence)
    index = coherence.index(max_value)
    model.append(n_topics(index))
    max_coherence.append(max_value)
    
# Save the result from the first step
#joblib.dump((model, max_coherence), '/project/biocomplexity/sdad/projects_data/ncses/prd/Dynamic_Topics_Modelling/nmf_fullabstract/first_stage.pkl' )
    

OSError: [Errno 12] Cannot allocate memory

In [5]:
# Plot the best model for each year and his coherence
year[0]

'2008'

In [ ]:
# Print topic from the first step
select_year = 2008

(nmf_time,topics_list,W_list,H_list) = joblib.load( path+'nmf_out/windows_nmf'+str(select_year)+'.pkl' )
k = model(year.index(select_year))
index = n_topics.index(k)
fy_topic_list = topics_list[index]
fy_topic_list